In [1]:
import os; import sys ; import matplotlib as mpl ; import matplotlib.pyplot as plt ; import numpy as np
import pathlib; from pathlib import Path; import pandas as pd; import flopy; from flopy.utils import MtListBudget

In [2]:
exe_name_mf = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf2005'
exe_name_mt = 'E:\\15_REPOS\\00_BETAMI\\bin\\mt3d-usgs_1.1.0_64'
current_directory = os.getcwd();  print(current_directory)

E:\15_REPOS\00_BETAMI\98_Plus\02_MT3D


In [3]:
path = pathlib.Path('E:\\15_REPOS\\00_BETAMI\\98_Plus\\02_MT3D')
path.mkdir(parents=True, exist_ok=True); os.chdir(path); current_directory = os.getcwd()
directory_name = 'ex-RES'    ; workspace = os.path.join('.', directory_name)
if os.path.isdir(workspace) is False:
    os.mkdir(workspace)                 
output_folder = Path('ex-RES\\p06_d') ; output_folder.mkdir(exist_ok=True)

In [4]:
def p06_d(dirname, mixelm, dt0):
    model_ws = os.path.join(workspace, dirname)
    nlay = 1 ;  nrow = 31 ;  ncol = 31 ;  delr = 900 ;  delc = 900 ;  delv = 20 ;  prsity = 0.30 ;  al = 100.0 ;  trpt = 1.0 ;  
    q0 = 86400.0 ;  c0 = 100.0 ;  perlen_mf = [912.5, 2737.5] ; perlen_mt = perlen_mf ; hk = 0.005 * 86400;     laytyp = 0
    nprs = 100  ; timprs = np.linspace(0, np.sum(perlen_mf), nprs, endpoint = False) ; timprs   # perlen_mf = [912.5, 2737.5] before extra

    modelname_mf = f"{dirname}_mf"
    mf = flopy.modflow.Modflow(modelname=modelname_mf, model_ws=model_ws, exe_name=exe_name_mf)
    dis = flopy.modflow.ModflowDis(mf,nlay=nlay,nrow=nrow,ncol=ncol,delr=delr,delc=delc,top=0.0,botm=[0 - delv],nper=2, perlen=perlen_mf)  # nper=2
    ibound = np.ones((nlay, nrow, ncol), dtype=int) * -1 ; ibound[ : ,1:nrow-1 ,1:ncol-1 ] = 1     ; strt = 0.0

    bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)
    lpf = flopy.modflow.ModflowLpf(mf, hk=hk, laytyp=laytyp)
    welspd = {0: [[0, 15, 15, q0]], 1: [[0, 15, 15, -q0]]}    # ___ injection / extraction, _____ steady state instantaneously
    wel = flopy.modflow.ModflowWel(mf, stress_period_data = welspd)
    sip = flopy.modflow.ModflowSip(mf)
    lmt = flopy.modflow.ModflowLmt(mf)                           ; mf.write_input()   ; mf.run_model(silent=True)

    modelname_mt = f"{dirname}_mt"
    mt = flopy.mt3d.Mt3dms(modelname=modelname_mt,model_ws=model_ws,exe_name=exe_name_mt,modflowmodel=mf)
    btn = flopy.mt3d.Mt3dBtn(mt,icbund=1,prsity=prsity,sconc=0,nper=2,perlen=perlen_mt,dt0=dt0,   obs=[(0, 15, 15)] , timprs=timprs ) # timprs
    dceps = 1.0e-5 ;  nplane = 1 ;  npl = 16 ;  nph = 16 ;  npmin = 4 ;  npmax = 32 ;  dchmoc = 1.0e-3 ;  nlsink = nplane; npsink = nph
    adv = flopy.mt3d.Mt3dAdv(mt,mixelm=mixelm,dceps=dceps,nplane=nplane,npl=npl,
                             nph=nph,npmin=npmin,npmax=npmax,nlsink=nlsink,npsink=npsink, percel=0.5)
    dsp = flopy.mt3d.Mt3dDsp(mt, al=al, trpt=trpt)               ; spd = {0: [0, 15, 15, c0, 2], 1: [0, 15, 15, 0.0, 2]}
    ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data = spd)
    gcg = flopy.mt3d.Mt3dGcg(mt)                                 ; mt.write_input()
    
    fname = os.path.join(model_ws, "MT3D001.UCN")
    if os.path.isfile(fname):
        os.remove(fname)
    mt.run_model(silent=True)
    fname = os.path.join(model_ws, "MT3D001.UCN")  ; ucnobj = flopy.utils.UcnFile(fname) ; conc = ucnobj.get_alldata()
    times = np.array(ucnobj.get_times())                    # times = ucnobj.get_times() 
    fname = os.path.join(model_ws, "MT3D001.OBS")
    if os.path.isfile(fname):
        cvt = mt.load_obs(fname)
    else:
        cvt = None
    fname = os.path.join(model_ws, "MT3D001.MAS")      ; mvt = mt.load_mas(fname)
    return mf, mt, conc, cvt, mvt, times

In [5]:
dt0= 56.5 ;  mf, mt, conc, cvt, mvt, times = p06_d("p06_d", -1, dt0)    # mixelm -1

In [21]:
conc1  = conc[:,0,:,:] ; save = conc1.reshape(conc1.shape[0],-1)  ; then=pd.DataFrame(save)  ; then.to_csv (output_folder / 'a_C.csv' )   
conc2  = conc[1]  ;   conc3 = conc2[0,:,:]  ; then = pd.DataFrame(conc3)                     ; then.to_csv (output_folder / 'b_C(x,y).csv')   
sa    = pd.DataFrame(times)                                                                  ;   sa.to_csv (output_folder / 'c_times.csv' )
sa    = pd.DataFrame(cvt)                                                                    ;   sa.to_csv (output_folder / 'd_cvt.csv' )
sa    = pd.DataFrame(mvt)                                                                    ;   sa.to_csv (output_folder / 'e_mvt.csv' )

In [7]:
# parameters (within fuction)
perlen=[912.5,2737.5] ;  nprs=100 ; nper=2  ; model_time = np.cumsum(perlen) ; timprs = np.linspace(0, np.sum(perlen), nprs, endpoint = True)
print('perlen    ',perlen), print('nper      ',nper) , print('nprs      ', nprs), print('model_time', model_time) # print('timprs    ', timprs)

perlen     [912.5, 2737.5]
nper       2
nprs       100
model_time [ 912.5 3650. ]


(None, None, None, None)

In [8]:
fname = os.path.join(output_folder, 'MT3D001.UCN')      
ucnobj = flopy.utils.UcnFile(fname)        ; conc = ucnobj.get_alldata()      ; times = ucnobj.get_times()  ; df = pd.DataFrame(times)
print('conc a_C_2_csv', conc.shape) , print('timprs = times', df)

conc a_C_2_csv (101, 1, 31, 31)
                               timprs = times           0
0       0.0
1      36.5
2      73.0
3     109.5
4     146.0
..      ...
96   3504.0
97   3540.5
98   3577.0
99   3613.5
100  3650.0

[101 rows x 1 columns]


(None, None)

In [22]:
mt_list = MtListBudget(output_folder/'p06_d_mt.list' )
gw_df, sw_df = mt_list.parse(start_datetime="01-01-2015")
sa = pd.DataFrame(gw_df)  ;   sa.to_csv (output_folder / 'f_Budget.csv' )